In [1]:
from IPython.display import clear_output  #use to clear output
from motor import XM430 , XL320

import traceback

import numpy as np
import pandas as pd
import matplotlib as mpl

import cv2
from cv2 import aruco
import pickle

import time
import keyboard

import math

In [7]:
#exception
class finished(Exception):
    """Base class for other exceptions"""
    pass

#Generator to serve moving positions for the xm430 motors step number of times starting from start location
def moveXm430(steps,start=0):
    change=0
    move=10
    
    #Set base postions
    dxl_home_position1=XM430.dxl1_home_position - start
    dxl_home_position2=XM430.dxl2_home_position + start
    print(f'base values: {[dxl_home_position1,dxl_home_position2]}')

    yield [dxl_home_position1,dxl_home_position2]
    #Move in the first direction
    for i in range(1,steps):
        dxl_goal_position1=dxl_home_position1 - move* i
        dxl_goal_position2=dxl_home_position2 - move* i
        change=yield [dxl_goal_position1,dxl_goal_position2]
        if change:
            break
    yield True
    #Move in the other direction
    for i in range(1,steps):
        dxl_goal_position1=dxl_home_position1 + move* i
        dxl_goal_position2=dxl_home_position2 + move* i
        change=yield [dxl_goal_position1,dxl_goal_position2]
        if change:
            break
    yield False#End of generator

#Generator to serve up step counts
def stepValues():
    steps=[40,40,40,40,40,40,40,40,40,40,40,50,50,50,50,50,55,55,55,55,55,55,55,60,60,60,60,60,65,65,65,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80]
    for value in steps:
        yield value//2 #Skip every second count. Adjust this for more dense data

#Increase the base starting positon of each trial
def increaseBasePositionXM430(basePosition,steps):
    basePosition+=40 #Amount to increment
    mover=moveXm430(steps,basePosition) #return a moving generator
    if basePosition>270: #Stop when over 270 as this is range of gripper
        raise finished
    return mover,basePosition

#Return the gripper to the center
def retrunHomeXM430(motors,basePosition):
    #calculate how far we have moved
    basePositions=[XM430.dxl1_home_position - basePosition, XM430.dxl2_home_position + basePosition]
    currentPosition=motors.getAllPositions()
    distance=currentPosition[0]-basePositions[0]
    positions=basePositions

    #iF short distance just move to base
    if abs(distance) <10:
        motors.writeGroupPosition(basePositions)
    else:
        #else move in num number of steps to get close
        num=20
        value=(abs(distance)-5)//num
        for j in range(1,value):
            direction= 1 if distance <0 else -1
            positions[0]=currentPosition[0]+direction*j*num
            positions[1]=currentPosition[1]+direction*j*num
            motors.writeGroupPosition(positions)
            time.sleep(.1)
        motors.writeGroupPosition(basePositions)

#Return the object slowly to the start postion. This reduces swinging of the object
def resetHome(motors,values):
    positions=values[::-5]
    positions.append(values[0])
    for base in positions:
        motors.writeGroupPosition([XM430.dxl1_home_position -base,XM430.dxl2_home_position +base])
        time.sleep(1)
    motors.writeGroupPosition([XM430.dxl1_home_position -values[0],XM430.dxl2_home_position +values[0]])

#Reset the grasp on the object
def resetObject(motors,values):
    motors.writeGroupPosition([XM430.dxl1_home_position,XM430.dxl2_home_position])
    time.sleep(1)
    motors.openGripper()
    motors.lowerObject()
    time.sleep(1)
    #Let the object hang and oscillations die down
    motors.raiseObject()
    time.sleep(3)
    motors.writeGroupPosition([XM430.dxl1_home_position,XM430.dxl2_home_position])
    time.sleep(1)
    motors.lowerObject()
    positions=values[0::10]
    #Move the object to the new starting postion to start the next trajectory
    for base in positions:
        motors.writeGroupPosition([XM430.dxl1_home_position -base,XM430.dxl2_home_position +base])
        time.sleep(1)
    motors.writeGroupPosition([XM430.dxl1_home_position -values[-1],XM430.dxl2_home_position +values[-1]])
    time.sleep(1)


    
# Checks if a matrix is a valid rotation matrix.
# code from https://learnopencv.com/rotation-matrix-to-euler-angles/
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

# Calculates rotation matrix to euler angles
#Used to convert rvec to euler angles
# code from https://learnopencv.com/rotation-matrix-to-euler-angles/
def rotationMatrixToEulerAngles(R) :

    assert(isRotationMatrix(R))

    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])

    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])

In [ ]:
#Creat a pandas data frame to get store the data
column_names=["motor 1", "motor 2", "X axis","y axis", "z Axis", "x rotation raw", "y rotation raw", "z rotation raw","x rotation", "y rotation", "z rotation", "Movement round"]
data=pd.DataFrame(columns = column_names)

#What is the chape and fingertips
shape=input("what is the shape")
tips=input("what are the finger tips")

#What com port and camera (if multiple)
xm430=XM430("COM4")
camera=0

#Counter used to not block camera when trying to slow motor commands
counter=0

#Sleep for motors
sleeptime=0.5

markerSize=2.5 # marker size in CM

# Open port and connect to the motors
result=xm430.initMotorControl()
if result != 0:
    print("error starting motors")
    quit()

# Enable Dynamixel motors and trun on the LED
xm430.writeVelocityLimit()
xm430.turnOnLED(1)
xm430.enableTorque()



#getthe calibration for the camera 
try:
    with open(f"truc{camera}.pckl", "rb") as f:
        calibrationParameters = pickle.load(f)

    cameraMatrix=calibrationParameters[1]
    distCoeffs=calibrationParameters[2]
except:
    print("need to calibrate")

aruco_dictionary = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)

# Create parameters to be used when detecting markers:
parameters = aruco.DetectorParameters_create()

cap = cv2.VideoCapture(camera,cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 450)

for i in range(3):
    #Set the first set of generators and other support parameters
    stepGenerator=stepValues()
    steps=next(stepGenerator)
    basePosition=0
    mover=moveXm430(steps,basePosition)
    values=[basePosition]
    nextPosition=0
    positions=next(mover)

    #Get a good intial grasp
    xm430.openGripper()

    time.sleep(1)
    xm430.raiseObject()
    time.sleep(3)
    xm430.writeGroupPosition(positions)
    time.sleep(1)
    xm430.lowerObject()
    Movement_round = 0

    try:
        while True:
            # Capture frame by frame from the video capture object 'capture':
            ret, frame = cap.read()
            # We convert the frame to grayscale:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # lists of ids and the corners beloning to each id# We call the function 'cv2.aruco.detectMarkers()'
            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray_frame, aruco_dictionary, parameters=parameters)

            # Draw detected markers:
            frame = aruco.drawDetectedMarkers(image=frame, corners=corners, borderColor=(0, 255, 0))


            # Display the resulting frame
            cv2.imshow('frame', frame)
        

            #When the motors have reached their stopped positions
            if not xm430.isMoving() and counter ==0:

                #If there are IDs then get the location of them
                if ids is not None:
                    
                    # rvecs and tvecs are the rotation and translation vectors respectively, for each of the markers in corners.
                    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, cameraMatrix, distCoeffs)
                    for rvec, tvec in zip(rvecs, tvecs):
                        aruco.drawAxis(frame, cameraMatrix, distCoeffs, rvec, tvec, 1)

                    #Get the position
                    centervector=tvecs[0][0]
                    rot=rvecs[0][0]
                    
                    #Convert the rvect to euler
                    rotation=cv2.Rodrigues(rvecs)
                    rotationvector=rotationMatrixToEulerAngles(rotation[0])
                else:
                    continue

                #Record data for the motors and the object
                currentPosition=xm430.getAllPositions()
                currentPosition.extend(centervector)
                currentPosition.extend(rot)
                currentPosition.extend(rotationvector)
                currentPosition.extend([Movement_round])
                #print(currentPosition)
                newdata=pd.Series(currentPosition,index=data.columns)
                data=data.append(newdata, ignore_index=True)



                #Code to move to the next location

                positions=next(mover)
                #Reached the end of the first trial.
                #Reset the object before starting the second trial
                if positions is True:
                    retrunHomeXM430(xm430,basePosition)
                    time.sleep(sleeptime)
                    resetHome(xm430,values)
                    time.sleep(sleeptime)
                    resetObject(xm430,values)
                    time.sleep(sleeptime)
                    positions=next(mover)
                    xm430.writeGroupPosition(positions)
                    time.sleep(sleeptime)
                
                #Reached the end of the second trial at that base
                #Return to the start position and get new generator for steps
                elif positions is False:
                    retrunHomeXM430(xm430,basePosition)
                    time.sleep(sleeptime)
                    resetHome(xm430,values)
                    time.sleep(sleeptime)
                    resetObject(xm430,values)
                    time.sleep(sleeptime)
                    
                    Movement_round+=1
                    steps=next(stepGenerator)
                    mover,basePosition = increaseBasePositionXM430(basePosition,steps)
                    values.append(basePosition)
                
                #Write positons
                else:
                    xm430.writeGroupPosition(positions)
                    
                counter +=1

                #Counter used to provide delay but not interfer with camera
            elif counter>=4:
                counter =0
            else:
                counter +=1
            
            #Break if q is pressed while the camera is working
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break
            
            #Break if at sign
            if keyboard.is_pressed("@"):
                break

    #BReak at end of the cycle
    except finished:
        print("finished loop")
    
    #Catch errors
    except:
        print(traceback.format_exc())
        # Disable Dynamixel Torque````
        print("exception")

    #Save the data
    finally:
        print("write dataS")
        filename=f"{shape} {tips} {i}"
        data.to_csv("..\\data\\under "+ filename+".csv")
        print(data.head())
        data=data[0:0]


#Release the object and close the camera and motor comunications

xm430.writeGroupPosition([XM430.dxl1_home_position,XM430.dxl2_home_position])
time.sleep(1)
xm430.shutdown()
# Release everything
cap.release()
cv2.destroyAllWindows()